In [1]:
#!/usr/bin/env python3
# import libraries
import cv2
import numpy as np
import os
import time
import tkinter as tk
from tkinter import filedialog, Toplevel, simpledialog, Label, Canvas, messagebox, Button
from PIL import Image, ImageTk
from tkinter import ttk

In [2]:
# create a class for the app
class VideoIndexerApp:
    # create the constructor
    def __init__(self, master):
        # set the master
        self.master = master
        # set the title
        self.master.title("Video Indexer")
        # set the geometry
        self.setup_ui()
    # create the ui
    def setup_ui(self):
        # create the style
        style = ttk.Style()
        # set the font
        style.configure("TButton", font=('Segoe UI', 10), padding=5)
        # set the padding
        style.configure("TLabel", font=('Segoe UI', 10))
        # set the frame with padding 10
        frame = ttk.Frame(self.master, padding="10")
        # set the grid
        frame.pack(fill=tk.BOTH, expand=True)
        # set the button upload video 
        self.load_video_button = ttk.Button(frame, text="Load Video", command=self.load_video)
        # set the Button grid
        self.load_video_button.grid(row=0, column=0, padx=10, pady=10, sticky="ew")
        # set generate index button
        self.generate_index_button = ttk.Button(frame, text="Generate Index Image", command=self.generate_index_image)
        self.generate_index_button.grid(row=1, column=0, padx=10, pady=10, sticky="ew")
        # set process all videos button
        self.process_all_videos_button = ttk.Button(frame, text="Process All Videos", command=self.process_all_videos)
        self.process_all_videos_button.grid(row=2, column=0, padx=10, pady=10, sticky="ew")
        # set add scaling button
        self.add_scaling_button = ttk.Button(frame, text="Add Scaling", command=self.add_scaling)
        self.add_scaling_button.grid(row=3, column=0, padx=10, pady=10, sticky="ew")
        # set index image label
        self.index_image_label = Label(frame)
        # set index image grid
        self.index_image_label.grid(row=4, column=0, padx=10, pady=10, sticky="nsew")
        # set the grid
        frame.columnconfigure(0, weight=1)
        for i in range(5):
            # set the grid
            frame.rowconfigure(i, weight=1)
    # load the video
    def load_video(self):
        # ask for the video
        # set the beggin time
        start_time = time.time()
        # set the video path
        self.video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
        # check if the video path is not none
        if self.video_path:
            messagebox.showinfo("Video Loaded", "Video successfully loaded!")
        # set the end time
        end_time = time.time()
        # print the time
        print(f"Load video time: {end_time - start_time:.2f} seconds")
    # generate the index image
    def generate_index_image(self):
        # check if the video path is not none
        if not self.video_path:
            # show the error
            messagebox.showerror("Error", "Load a video first!")
            return
        # set the line y
        line_y = simpledialog.askinteger("Input", "Enter the y-coordinate of the line to extract:", minvalue=0)
        # check if the line y is not none
        if line_y is not None:
            # set the beggin time#
            start_time = time.time()
            # create the index image
            self.index_image, self.frames_positions, _ = create_video_index(self.video_path, line_y)
            # set the end time
            end_time = time.time()
            #check if the index image is not none
            if self.index_image is not None:
                # set the photo
                photo = ImageTk.PhotoImage(image=Image.fromarray(self.index_image))
                # set the image
                self.index_image_label.config(image=photo)
                # bind the image
                self.index_image_label.image = photo
                # bind the image
                self.index_image_label.bind("<Button-1>", self.on_click)
                # print the time
            print(f"Generate index image time: {end_time - start_time:.2f} seconds")
    # play the video from the frame
    def on_click(self, event):
        # set the  y
        y = event.y
        # set the frame index
        frame_index = self.frames_positions[y] if y < len(self.frames_positions) else None
        # check if the frame index is not none
        if frame_index is not None:
            # play the video from the frame
            self.play_video_from_frame(frame_index)
    # play the video from the frame
    def play_video_from_frame(self, frame_index):
        # open the video
        cap = cv2.VideoCapture(self.video_path)
        # check if the cap is not none
        if not cap.isOpened():
            # show the error
            messagebox.showerror("Error", "Error opening video stream or file")
            # return
            return
        # set the ret
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        # read the frame 
        ret, frame = cap.read()
        # check if the ret is not none
        if not ret:
            # show the error
            messagebox.showerror("Error", "Could not read the frame from video.")
            return
        # set the cv window
        cv_window = Toplevel(self.master)
        # set the title
        cv_window.title(f"Playing from Frame {frame_index}")
        # set the Width
        frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        # set the Height
        frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        # set the cv canvas
        cv_canvas = Canvas(cv_window, width=int(frame_width), height=int(frame_height))
        # set the pack
        cv_canvas.pack()
        # set the stop button
        stop_button = Button(cv_window, text="Stop", command=cv_window.destroy)
        # stop button  pack
        stop_button.pack()
        # set the ret
        while ret:
            # convert colour to rgb 
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # convert to image
            frame = Image.fromarray(frame)
            # convert to photo
            photo = ImageTk.PhotoImage(image=frame)
            # set the image
            cv_canvas.create_image(0, 0, anchor=tk.NW, image=photo)
            # set the image
            cv_canvas.image = photo
            # set the update
            cv_window.update_idletasks()
            # set the update
            cv_window.update()
            # read the frame
            ret, frame = cap.read()
            # check wait Key
            if cv2.waitKey(28) & 0xFF == ord('q'):
                break
        # release the cap
        cap.release()
        cv_window.destroy()
    # add scaling
    def add_scaling(self):
        # set the scale
        scale = simpledialog.askfloat("Set Frame Scaling", "Enter frame scale factor (0.01 - 1.0):", minvalue=0.01, maxvalue=1.0, initialvalue=0.1)
        # check if the scale is not none
        if scale:
            # set the scale
            messagebox.showinfo("Scaling Set", f"Scaling set to: {scale}")
    # process all videos
    def process_all_videos(self):
        # ask for the directory
        directory = filedialog.askdirectory()
        # check if the directory is not none
        if directory:
            # set the line y
            line_y = simpledialog.askinteger("Input", "Enter the y-coordinate of the line to extract:", minvalue=0)\
            # check if the line y is not none
            for file in os.listdir(directory):
                # check if the file is a video
                if file.lower().endswith((".mp4", ".avi", ".mov")):
                    # set the beggin time
                    video_path = os.path.join(directory, file)
                    # set the beggin time
                    start_time = time.time()
                    # create the index image
                    index_image, _, _ = create_video_index(video_path, line_y)
                    # set the end time
                    end_time = time.time()
                    # check if the index image is not none
                    if index_image is not None:
                        # output path
                        output_path = os.path.join(directory, f"{os.path.splitext(file)[0]}_index.png")
                        # image writer and make type
                        cv2.imwrite(output_path, index_image)
                        # save the image
                        print(f"Saved index image to {output_path}")
                        # print the time
                    print(f"Processed {file} in {end_time - start_time:.2f} seconds")

In [3]:

# create the index image
def create_video_index(video_path, line_y):
    # set the beggin time
    start_time = time.time()
    # open the video
    cap = cv2.VideoCapture(video_path)
    # check if the cap is not none
    if not cap.isOpened():
        return None, None, None
    # set the index image
    index_image, frame_positions = [], []
    # set the loop
    while True:
        # read the frame
        ret, frame = cap.read()
        # check if the ret is not none
        if not ret:
            break
        # check if the line y is not none
        if line_y >= frame.shape[0]:
            continue
        # set the line
        line = frame[line_y:line_y+1, :, :]
        # append the line
        index_image.append(line)
        # append the frame position
        frame_positions.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
    # set the index image
    index_image = np.concatenate(index_image, axis=0) if index_image else None
    # release the cap
    cap.release()
    # set the end time
    end_time = time.time()
    return index_image, frame_positions, end_time - start_time



In [5]:
# main
def main():
    # set the root
    root = tk.Tk()
    # set the title
    app = VideoIndexerApp(root)
    root.mainloop()
# call the main
if __name__ == "__main__":
    main()


Load video time: 4.27 seconds
Generate index image time: 0.35 seconds


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\asus\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\asus\AppData\Local\Temp\ipykernel_5568\3376518867.py", line 97, in on_click
    self.play_video_from_frame(frame_index)
  File "C:\Users\asus\AppData\Local\Temp\ipykernel_5568\3376518867.py", line 142, in play_video_from_frame
    cv_canvas.create_image(0, 0, anchor=tk.NW, image=photo)
  File "C:\Users\asus\anaconda3\Lib\tkinter\__init__.py", line 2846, in create_image
    return self._create('image', args, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\asus\anaconda3\Lib\tkinter\__init__.py", line 2832, in _create
    return self.tk.getint(self.tk.call(
                          ^^^^^^^^^^^^^
_tkinter.TclError: invalid command name ".!toplevel.!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\asus\anaconda3\Lib\t